In [ ]:
from flask import Flask, request, jsonify
import mysql.connector
from mysql.connector import errorcode
from datetime import datetime

#  Flask app setup 
# This is a simple Flask application that ingests system and process metrics from agents and stores them in a MySQL database.
app = Flask(__name__)   

# Base DB credentials
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'root',
}

AUTH_TOKEN = "psg_xdr_agent_token"

def create_database(cursor, db_name):
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS `{db_name}` DEFAULT CHARACTER SET 'utf8'")

def sanitize_db_name(name: str) -> str:
    """Removes problematic characters from system names for MySQL DB"""
    return name.strip().lower().replace('-', '_').replace('.', '_')

@app.route("/ingest", methods=["POST"])
def ingest():
    try:
        #  1. Authorization
        auth_header = request.headers.get("Authorization")
        if auth_header != f"Bearer {AUTH_TOKEN}":
            return jsonify({"status": "error", "message": "Unauthorized"}), 401

        #  2. Get data
        data = request.get_json()
        if not data:
            return jsonify({"status": "error", "message": "No JSON payload received"}), 400

        #  3. Prepare timestamp & DB name
        hostname = data.get("hostname", "unknown_host")
        db_name = sanitize_db_name(hostname) + "_db"

        print(f" Receiving data for agent: {hostname}, DB: {db_name}")

        timestamp = data.get("timestamp", datetime.utcnow().timestamp())
        timestamp = datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

        #  4. Connect & create the agent-specific database if not exists
        conn = mysql.connector.connect(**db_config)
        cursor = conn.cursor()
        create_database(cursor, db_name)
        conn.commit()
        cursor.close()
        conn.close()

        #  5. Connect again WITH the new DB in order to create tables
        db_config_with_db = db_config.copy()
        db_config_with_db['database'] = db_name
        conn = mysql.connector.connect(**db_config_with_db)
        cursor = conn.cursor()

        #  6. Create system_metrics table (if not exists) and insert data
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS system_metrics (
                id INT AUTO_INCREMENT PRIMARY KEY,
                timestamp DATETIME,
                memory_total FLOAT,
                memory_used FLOAT,
                memory_available FLOAT,
                memory_percent FLOAT,
                cpu_percent FLOAT,
                disk_total FLOAT,
                disk_used FLOAT,
                disk_free FLOAT,
                disk_percent FLOAT,
                network_bytes_sent BIGINT,
                network_bytes_recv BIGINT
            )
        ''')

        #  7. Insert system metrics
        cursor.execute('''
            INSERT INTO system_metrics (
                timestamp, memory_total, memory_used, memory_available, memory_percent,
                cpu_percent, disk_total, disk_used, disk_free, disk_percent,
                network_bytes_sent, network_bytes_recv
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ''', (
            timestamp,
            data.get("memory_total", 0), data.get("memory_used", 0),
            data.get("memory_available", 0), data.get("memory_percent", 0),
            data.get("cpu_percent", 0),
            data.get("disk_total", 0), data.get("disk_used", 0),
            data.get("disk_free", 0), data.get("disk_percent", 0),
            data.get("network_bytes_sent", 0), data.get("network_bytes_recv", 0)
        ))

        #  8. Create process_metrics table (if not exists)
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS process_metrics (
                id INT AUTO_INCREMENT PRIMARY KEY,
                agent_id VARCHAR(255),
                timestamp DATETIME,
                pid INT,
                process_name VARCHAR(255),
                username VARCHAR(255),
                cpu_percent FLOAT,
                memory_percent FLOAT,
                cmdline TEXT,
                open_file TEXT,
                is_suspicious TINYINT(1)
            )
        ''')

        #  9. Insert process data
        insert_process_query = '''
            INSERT INTO process_metrics (
                agent_id, timestamp, pid, process_name, username,
                cpu_percent, memory_percent, cmdline, open_file, is_suspicious
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        '''

        for proc in data.get("processes", []):
            proc_ts = proc.get("timestamp", datetime.utcnow().timestamp())
            proc_ts = datetime.fromtimestamp(proc_ts).strftime('%Y-%m-%d %H:%M:%S')
            is_suspicious = 1 if proc.get("is_suspicious", False) else 0

            cursor.execute(insert_process_query, (
                proc.get("agent_id"),
                proc_ts,
                proc.get("pid"),
                proc.get("process_name"),
                proc.get("username"),
                proc.get("cpu_percent"),
                proc.get("memory_percent"),
                proc.get("cmdline"),
                proc.get("open_file"),
                is_suspicious
            ))

        conn.commit()
        cursor.close()
        conn.close()

        print(f"Data ingested into database: {db_name}")
        return jsonify({"status": "success", "message": f"Data ingested into {db_name}"}), 200

    except Exception as e:
        print(" Server Error:", str(e))
        return jsonify({"status": "error", "message": str(e)}), 500

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000, debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.29.9:5000
Press CTRL+C to quit


 Receiving data for agent: LAPTOP-LGQ3OFRV, DB: laptop_lgq3ofrv_db


C:\Users\abhinav\AppData\Local\Temp\ipykernel_13700\2767517858.py:43: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = data.get("timestamp", datetime.utcnow().timestamp())
C:\Users\abhinav\AppData\Local\Temp\ipykernel_13700\2767517858.py:122: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  proc_ts = proc.get("timestamp", datetime.utcnow().timestamp())
192.168.29.9 - - [02/Jun/2025 19:07:40] "POST /ingest HTTP/1.1" 200 -


Data ingested into database: laptop_lgq3ofrv_db
 Receiving data for agent: LAPTOP-LGQ3OFRV, DB: laptop_lgq3ofrv_db


...

Data ingested into database: laptop_lgq3ofrv_db
 Receiving data for agent: LAPTOP-LGQ3OFRV, DB: laptop_lgq3ofrv_db


192.168.29.9 - - [02/Jun/2025 19:15:48] "POST /ingest HTTP/1.1" 500 -


 Server Error: 1406 (22001): Data too long for column 'open_file' at row 1
 Receiving data for agent: LAPTOP-LGQ3OFRV, DB: laptop_lgq3ofrv_db


192.168.29.9 - - [02/Jun/2025 19:17:47] "POST /ingest HTTP/1.1" 200 -


Data ingested into database: laptop_lgq3ofrv_db
